In [1]:
from dotenv import load_dotenv
envs = load_dotenv('../.env', override=True)
import os

In [3]:
from anthropic import Anthropic
from litellm import completion_with_retries, completion, acompletion
# from langchain_anthropic import ChatAnthropic
from openai import AsyncOpenAI
from time import sleep
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [5]:
llm = ChatAnthropic(name='anthro', 
                    model_name="claude-3-sonnet-20240229",
                    temperature=1.0, 
                    anthropic_api_key=os.environ['ANTHROPIC_API_KEY'])

In [7]:
answer = llm.invoke("how can langsmith help with testing")

In [4]:
USER_QUESTION="What show won the Outstanding Drama award at the 2024 Emmys?"

In [5]:
GENERATE_QUERIES=f"""Human: You are an expert at generating search queries for the Brave search engine.
Generate three search queries that are relevant to this question. Output only valid JSON.

User question: {USER_QUESTION}

Format: {{"queries": ["query_1", "query_2", "query_3"]}}\n\nAssistant: {{"""

In [6]:
print(GENERATE_QUERIES)

Human: You are an expert at generating search queries for the Brave search engine.
Generate three search queries that are relevant to this question. Output only valid JSON.

User question: What show won the Outstanding Drama award at the 2024 Emmys?

Format: {"queries": ["query_1", "query_2", "query_3"]}

Assistant: {


In [7]:
client = Anthropic()

In [12]:
messages = [
                {"role": "user", "content": GENERATE_QUERIES},
            ]

In [13]:
message = client.messages.create(model="claude-3-haiku-20240307", 
                                    temperature=1.0, 
                                    max_tokens=500,
                                    messages=messages)

In [18]:
message.usage.input_tokens, message.usage.output_tokens

(88, 50)

In [27]:
lite_message = completion(model="claude-3-haiku-20240307",
           messages=messages,
           temperature=1.0,
           max_tokens=200)

In [29]:
lite_message.dict()

{'id': 'chatcmpl-bafd81ac-8579-437d-b030-ecff4a1ed14b',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '"queries": [\n  "2024 Emmy awards outstanding drama winner",\n  "top drama series emmy 2024",\n  "most awarded drama series 2024 emmys"\n]\n\n}',
    'role': 'assistant'}}],
 'created': 1712030502,
 'model': 'claude-3-haiku-20240307',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'usage': {'prompt_tokens': 88, 'completion_tokens': 49, 'total_tokens': 137}}

In [46]:
context1 = '''
Another benefit of gRPC is its enhanced type safety, achieved through the explicit definition of data structures and types in protobufs. This approach significantly reduces common challenges encountered when working with GraphQL and JSON, such as ambiguity in data types and difficulties in ensuring readability.
In the context of JSON and GraphQL, determining the correct data type for numbers (e.g., distinguishing between integers and floating-point numbers) or interpreting empty properties can be problematic. In Weaviate, this can lead to AutoSchema inferring inappropriate data types, potentially leading to data integrity issues.
Moreover, parsing complex and deeply nested JSON responses, or crafting intricate GraphQL queries, can degrade the developer experience and elevate the risk of errors. This is also partly due to having to conform to the structure of the GraphQL schema. This has been a challenge for certain tasks, such as implementing GroupBy queries and responses in Weaviate.
gRPC addresses these issues head-on by facilitating the customization of data structures for both requests and responses. Its reliance on strictly defined data types streamlines parsing processes and boosts reliability.
'''

context2 = '''
Vectors are a great way to represent meaning. Vectors are arrays of elements that can capture meaning from different data types, such as texts, images, videos, and other content. The elements are called dimensions. High dimension vectors capture more information, but they are harder to work with.
Vector databases make it easier to work with high dimensional vectors. Consider search; Vector databases efficiently measure semantic similarity between data objects. When you run a similarity search, a vector database like Weaviate uses a vectorized version of the query to find objects in the database that have vectors similar to the query vector.
Vectors are like coordinates in a multi-dimensional space. A very simple vector might represent objects, words in this case, in a 2-dimensional space.
In the graph below, the words Apple and Banana are shown close to each other. Newspaper and Magazine are also close to each other, but they are far away from Apple and Banana in the same vector space.
Within each pair, the distance between words is small because the objects have similar vector representations. The distance between the pairs is larger because the difference between the vectors is larger. Intuitively, fruits are similar to each other, but fruits are not similar to reading material.
'''

In [48]:
prompts = [context1, context2]
messages = [{"role": "system", "content": "You are a highly experienced data annotator.  Your job is to create two questions that can be answered from the provided context."},
            {"role": "assistant", "content": context}]

In [49]:
# cage = []
# for chunk in completion(model='claude-3-haiku-20240307', messages=messages, temperature=1.0, stream=True):
#     cage.append(chunk['choices'][0]['delta']['content'].strip())
#     if any(cage):
#         print(' '.join(cage))
#         sleep(0.05)

In [56]:
async def async_task(message: list[dict]):
    response = await acompletion(model="gpt-3.5-turbo-1106", messages=message, temperature=1.0)
    return response

In [64]:
%%time
async def gather(prompts: list[str]):
    tasks = []
    for p in prompts:
        messages = [{"role": "system", "content": "You are a highly experienced data annotator.  Your job is to create two questions that can be answered from the provided context."},
                    {"role": "assistant", "content": prompt.format(context=p)}]
        tasks.append(async_task(messages))
    asyncio.gather(*tasks)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 6.91 µs


In [65]:
%%time
responses = asyncio.run(gather(prompts))

CPU times: user 328 µs, sys: 109 µs, total: 437 µs
Wall time: 416 µs


In [63]:
responses